In [8]:
# Following https://www.tensorflow.org/get_started/mnist/pros
#

# load data
#
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [9]:
# Recap of the simpler MNIST tutorial for beginners
#
import tensorflow as tf
sess = tf.InteractiveSession()

# x = inputs, y_ = targets
#
x = tf.placeholder(tf.float32, shape=[None, 784])
y_ = tf.placeholder(tf.float32, shape=[None, 10])

# model parameters
#
W = tf.Variable(tf.zeros([784,10]))
b = tf.Variable(tf.zeros([10]))

# Initialize all vars -- IMPORTANT!
#
sess.run(tf.global_variables_initializer())

# model: prediction y = softmax(x W + b)
#
y = tf.matmul(x,W) + b

# loss function = cross-entropy between y (prediction) and y_ (target)
#
cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y))

# train model using gradient descent
#
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

# Run 1000 training batches of size 100 random samples
#
for _ in range(1000): 
    batch = mnist.train.next_batch(100) 
    train_step.run(feed_dict={x: batch[0], y_: batch[1]})
    
# Evaluate model
#
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels})

0.9167


In [12]:
# Getting advanced....
#

# ReLU neurons -> initialize weights with slightly positive bias to avoid "dead" neurons
# also random to break symmetries
#
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

# initialize biases to 0.1
#
def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

# convolution layers
#
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

# pooling layers
#
def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

#
# Now we build the CNN
#

# First layer = convolution (32 features for each 5x5 patch) + max pooling
# 
W_conv1 = weight_variable([5, 5, 1, 32]) # 5 x 5 patch ==> 32 dim features output
b_conv1 = bias_variable([32])

# reshape x too 
#
x_image = tf.reshape(x, [-1, 28, 28, 1])

# convolve x * W, add bias, apply ReLU f'n, max pool => reduces image to 14x14
#
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

# Second layer = 64 features for each 5x5 patch
#
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2) # reduce image to 7x7

# Densely connected layer with 1024 neurons to process entire image
#
W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

# apply dropout to reduce overfitting
#
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

# readout layer
#
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2

In [14]:
# Do the actual training
# use ADAM instead of steepest gradient descent
# parameters for dropping
# log every 100th iteration
#
# WARNING: takes forever...
#
cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer()) # IMPORTANT -- init vars
    for i in range(20000):
        batch = mnist.train.next_batch(50) # pick 50 random samples
        if i % 100 == 0: # print every now and then
            train_accuracy = accuracy.eval(feed_dict={
                x: batch[0], y_: batch[1], keep_prob: 1.0})
            print('step %d, training accuracy %g' % (i, train_accuracy))
        # next training batch
        #
        train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})

    print 'test accuracy %g' % accuracy.eval(feed_dict={
        x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0})

step 0, training accuracy 0.04
step 100, training accuracy 0.84
step 200, training accuracy 0.86


KeyboardInterrupt: 

In [26]:
# print size of input vector LOL the agony

import numpy as np

sess = tf.Session()
print sess.run(tf.sqrt(tf.cast(tf.shape(x)[1], dtype='double')), feed_dict={
        x: mnist.test.images})

28.0
